# Hands-on Lab: Building an Agent System with Databricks

**Please use Serverless Compute.**

## Part 1 - Design Your First Agent
This first agent will follow the workflow of a customer service representative and explain various agent functions.
We will focus on processing product returns and go through the specific steps.

### 1.1 Create Simple Tools
- **SQL Functions**: Create queries to access data important for each step of the return processing workflow.
- **Simple Python Functions**: Create and register Python functions to overcome common limitations of language models.

### 1.2 Integration with LLM [AI Playground]
- Combine the created tools with a language model (LLM) and use them in the AI Playground.

### 1.3 Test the Agent [AI Playground]
- Ask questions to the agent and observe its responses.
- Use MLflow traces to further explore the agent's performance.

In [0]:
# Install Libraries
%pip install -qqqq -U -r requirements.txt
# Restart to load packages into the Python environment
dbutils.library.restartPython()

In [0]:
%run ../config

In [0]:
# Parameter Settings
from databricks.sdk import WorkspaceClient
import os
import re

# Get information about the current user using the workspace client
w = WorkspaceClient()
user_email = w.current_user.me().emails[0].value
username = user_email.split('@')[0]
username = re.sub(r'[^a-zA-Z0-9_]', '_', username) # Replace special characters with underscores

# Specify schema
user_schema_name = f"agents_lab_{username}" # Per-user schema

print("Your catalog is:", catalog_name)
print("Your schema is:", user_schema_name)

# Create schema
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{user_schema_name}")

# Customer Service Return Processing Workflow

Below is a structured overview of the main steps that a customer service representative typically follows when **processing a return**. This workflow ensures consistency and clarity across the support team.

---

## 1. Retrieve the Latest Return in the Processing Queue
- **Action**: Identify and retrieve the latest return request from the ticketing or return system.  
- **Reason**: Ensures you are addressing the most urgent or next-in-line customer issue.

---

In [0]:
# Display catalog name
query = f"SELECT '{catalog_name}' as catalog_name"
df = spark.sql(query)
display(df)

In [0]:
# Retrieve Latest Return in Processing Queue
# Retrieve the latest return
query = (
    f"SELECT "
    f"  cast(date_time as date) as case_time, "
    f"  issue_category, "
    f"  issue_description, "
    f"  name "
    f"FROM {catalog_name}.{system_schema_name}.cust_service_data "
    f"ORDER BY date_time DESC "
    f"LIMIT 1"
)
df = spark.sql(query)
display(df)

In [0]:
# Set catalog and schema
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {system_schema_name}")

In [0]:
# Create function to get the latest return
query = f'''
CREATE OR REPLACE FUNCTION 
  {catalog_name}.{user_schema_name}.get_latest_return()
RETURNS TABLE(purchase_date DATE, issue_category STRING, issue_description STRING, name STRING)
COMMENT 'Returns the latest customer service response (such as returns).'
RETURN (
  SELECT 
    CAST(date_time AS DATE) AS purchase_date,
    issue_category,
    issue_description,
    name
  FROM {catalog_name}.{system_schema_name}.cust_service_data
  ORDER BY date_time DESC
  LIMIT 1
);
'''
spark.sql(query)

In [0]:
# Test the function to get the latest return
query = f"SELECT * FROM {catalog_name}.{user_schema_name}.get_latest_return()"
df = spark.sql(query)
display(df)

---

## 2. Retrieve Company Policy
- **Action**: Access internal knowledge bases or policy documents regarding returns, refunds, and exchanges.  
- **Reason**: Ensures compliance with company guidelines, preventing potential errors or conflicts.

---

In [0]:
# Create function to retrieve return policy
query = f'''
CREATE OR REPLACE FUNCTION
  {catalog_name}.{user_schema_name}.get_return_policy()
RETURNS TABLE (
  policy           STRING,
  policy_details   STRING,
  last_updated     DATE
)
COMMENT 'Returns details of the return policy.'
LANGUAGE SQL
RETURN (
  SELECT
    policy,
    policy_details,
    last_updated
  FROM {catalog_name}.{system_schema_name}.policies
  WHERE policy = 'Return Policy'
  LIMIT 1
);
'''
spark.sql(query)

In [0]:
# Test Function to Retrieve Return Policy
query = f"SELECT * FROM {catalog_name}.{user_schema_name}.get_return_policy()"
df = spark.sql(query)
display(df)

---

## 3. Retrieve the UserID of the Latest Return
- **Action**: Record the unique identifier of the user from the return request details.  
- **Reason**: Efficiently process and avoid confusion in customer records by referencing the correct user data.

---

In [0]:
# Create Function to Retrieve UserID Based on Name
query = f'''
CREATE OR REPLACE FUNCTION
  {catalog_name}.{user_schema_name}.get_user_id(user_name STRING)
RETURNS STRING
COMMENT 'This function takes the customer name as input and returns the corresponding user ID.'
LANGUAGE SQL
RETURN 
SELECT customer_id 
FROM {catalog_name}.{system_schema_name}.cust_service_data 
WHERE name = user_name
LIMIT 1
;
'''
spark.sql(query)

In [0]:
# Test Function to Retrieve UserID Based on Name
user_name = "Nicolas Pelaez"
query = f"SELECT {catalog_name}.{user_schema_name}.get_user_id('{user_name}') as user_id"
df = spark.sql(query)
display(df)

---

## 4. Query Order History Using UserID
- **Action**: Search the order management system or customer database using the UserID.  
- **Reason**: By checking past purchase history, return trends, and special notes, you can determine the appropriate next steps (e.g., eligibility for return).

---

In [0]:
# Create function to retrieve order history based on userID
query = f'''
CREATE OR REPLACE FUNCTION
  {catalog_name}.{user_schema_name}.get_order_history(user_id STRING)
RETURNS TABLE (returns_last_12_months INT, issue_category STRING)
COMMENT 'This function takes the customer user_id as input and returns the number of returns and issue categories for the past 12 months.'
LANGUAGE SQL
RETURN 
SELECT count(*) as returns_last_12_months, issue_category 
FROM {catalog_name}.{system_schema_name}.cust_service_data 
WHERE customer_id = user_id 
GROUP BY issue_category;
'''
spark.sql(query)

In [0]:
# Test Function to Retrieve Order History Based on UserID
user_id = "453e50e0-232e-44ea-9fe3-28d550be6294"
query = f"SELECT * FROM {catalog_name}.{user_schema_name}.get_order_history('{user_id}')"
df = spark.sql(query)
display(df)

---

## 5. Provide a Python Function to LLM to Get Today's Date
- **Action**: Prepare a **Python function** that can provide the current date to the LLM (Large Language Model).  
- **Reason**: Automatically obtaining the date makes it easier to manage scheduling pickups, refund timelines, and contact deadlines.

###### Note: By using functions registered with System.ai.python_exec, you can execute code generated by the LLM in a sandboxed environment.
---

In [0]:
# Very Simple Python Function
def get_todays_date() -> str:
    """
    Returns today's date in 'YYYY-MM-DD' format.

    Returns:
        str: Returns today's date in 'YYYY-MM-DD' format.
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

In [0]:
# Test Python Function
today = get_todays_date()
today

In [0]:
# Register Python Function in Unity Catalog
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# Deploy this tool to UC, and automatically set UC metadata based on the tool's docstring and type hints
python_tool_uc_info = client.create_python_function(func=get_todays_date, catalog=catalog_name, schema=user_schema_name, replace=True)

# The tool is deployed as a function named `{catalog}.{schema}.{func}` in UC. Here, {func} is the function name
# Display the deployed Unity Catalog function name
display(f"Deployed Unity Catalog function name: {python_tool_uc_info.full_name}")

In [0]:
# View Created Function
from IPython.display import display, HTML

# Get Databricks workspace host URL
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# Create HTML link to the created function
html_link = f'<a href="https://{workspace_url}/explore/data/functions/{catalog_name}/{user_schema_name}/get_todays_date" target="_blank">Check the registered function in Unity Catalog</a>'
display(HTML(html_link))

## Now, let's see how to assemble the first agent using these functions in the AI Playground!

- **System Prompt**: `Call tools until you are confident that all internal policies are satisfied.`
- **Example Question**: `Based on our company policy, should we accept the latest return?`

### You can find the AI Playground from "AI/ML" in the left navigation bar, or use the link created below.

In [0]:
# Create HTML link to AI Playground
html_link = f'<a href="https://{workspace_url}/ml/playground" target="_blank">Go to AI Playground</a>'
display(HTML(html_link))